In [2]:
pip install keras-tuner

     |████████████████████████████████| 102kB 3.0MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=cd0187bd4580009d824590a69f909d3ca8674286d12df2c9aa2a94fb7d116a23
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [16]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [17]:
df = pd.read_csv('/content/sample_data/Real_Combine.csv')
df.tail()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667
1092,16.3,23.4,9.0,1017.3,68.0,1.3,7.8,18.3,185.583333


In [18]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
x.shape,y.shape

((1093, 8), (1093,))

In [26]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [29]:
tuner = RandomSearch(build_model, objective='val_mean_absolute_error',max_trials=5,executions_per_trial=3, directory='project2', project_name='Air Quality Index2')

In [22]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [30]:
tuner.search(x_train,y_train, epochs=5, validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 10s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 36s
INFO:tensorflow:Oracle triggered exit


In [31]:
tuner.results_summary()

Results summary
Results in project2/Air Quality Index2
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 3
units_0: 96
units_1: 96
learning_rate: 0.01
units_2: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 4
units_0: 288
units_1: 224
learning_rate: 0.001
units_2: 512
units_3: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 4
units_0: 320
units_1: 352
learning_rate: 0.0001
units_2: 128
units_3: 160
Score: nan
Trial summary
Hyperparameters:
num_layers: 11
units_0: 192
units_1: 416
learning_rate: 0.001
units_2: 352
units_3: 352
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 7
units_0: 480
units_1: 256
learning_rate: 0.0001
units_2: 384
units_3: 384
units_4: 128
units_5: 288
units_6: 416
units_7: 128
units_8: 64
units_9: 288
units_10: 448
Score: nan
